In [7]:
import os
import shutil
import pandas as pd
import pyreadstat
import numpy as np

In [8]:
data, meta = pyreadstat.read_sav('Dr.miratashi_3 copy.sav')

In [9]:
def move_folders(folder_list, source_directory, destination_directory):
    for folder_name in folder_list:
        source_path = os.path.join(source_directory, folder_name)
        destination_path = os.path.join(destination_directory, folder_name)
        shutil.move(source_path, destination_path)
        print(f"Moved folder '{folder_name}' from '{source_directory}' to '{destination_directory}'")

In [10]:
def create_pid_folder(parent_directory, pid):
    for folder_name in pid:
        folder_path = os.path.join(parent_directory, str(folder_name))
        os.makedirs(folder_path, exist_ok=True)

        # Create the "FGT" and "BPE" sub folders within each folder
        fgt_folder_path = os.path.join(folder_path, 'FGT')
        bpe_folder_path = os.path.join(folder_path, 'BPE')
        os.makedirs(fgt_folder_path, exist_ok=True)
        os.makedirs(bpe_folder_path, exist_ok=True)

In [11]:
def automated_copy(origin_directory, destination_directory, pid, fgt_idx, bpe_idx):
    img_folder_names = []
    for patient_id in pid:
        dcm_directory = origin_directory + str(patient_id) + '/IMAGE/DCM/'
        for i, (root, dirs, files) in enumerate(os.walk(dcm_directory)):
            if i == 0:
                img_folder_names.append(dcm_directory + str(dirs[i]))
            
    for i, patient_id in enumerate(pid):
        start_fgt, end_fgt = fgt_idx[i]
        start_bpe, end_bpe = bpe_idx[i]
        destination_fgt_folder = destination_directory + str(patient_id) + '/FGT'
        destination_bpe_folder = destination_directory + str(patient_id) + '/BPE'
        for j in range(start_fgt, end_fgt+1):
            if len(str(j)) == 4:
                fgt_img_directory = img_folder_names[i] + '/I000' + str(j)
            else:
                fgt_img_directory = img_folder_names[i] + '/I0000' + str(j)
            shutil.copy(fgt_img_directory, destination_fgt_folder)
            
        for j in range(start_bpe, end_bpe+1):
            if len(str(j)) == 4:
                bpe_img_directory = img_folder_names[i] + '/I000' + str(j)
            else:
                bpe_img_directory = img_folder_names[i] + '/I0000' + str(j)
            shutil.copy(bpe_img_directory, destination_bpe_folder)

In [12]:

def get_folder_names(path, folders):
    for entry in os.scandir(path):
        if entry.is_dir():
            folders.append(entry.name)
    return folders



# Data Preprocessing

## Step 1

### At this step, we get the unique patient IDs and remove data with multiple diagnosis.


In [13]:
folder_names = []

mpath = 'E:/Breast Data/Breast/'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'E:/Breast Data/breast newNAC'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/01/sps'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/02/sps'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/03/breast new'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/03/breast new 2'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/03/nac'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/03/sps'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'G:/Breast Data/nac'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'G:/Breast Data/sps'
folder_names = get_folder_names(mpath, folder_names)


In [14]:
folder_names = np.array(folder_names)

In [15]:
folder_names_unique = np.unique(folder_names)

In [16]:
data_codes = np.array(data.code)
data_codes_unique = np.unique(data_codes)

In [17]:
same_codes_unique = []
for i in folder_names_unique:
    if i in data_codes_unique:
        same_codes_unique.append(i)
        
same_codes_unique = np.array(same_codes_unique)

In [18]:
not_in_dataset = []
for i in data_codes_unique:
    if i not in same_codes_unique:
        not_in_dataset.append(i)

not_in_dataset_unique = np.unique(np.array(not_in_dataset))

In [19]:
same_codes_unique_mdd_removed = []
for i in same_codes_unique:
    if np.array(data[data.code == i].FG).shape[0] > 1:
        fg_i = np.array(data[data.code == i].FG)
        bpe_i = np.array(data[data.code == i].BPE)
        if np.all(fg_i==fg_i[0]) and np.all(bpe_i==bpe_i[0]):
            same_codes_unique_mdd_removed.append(i)
    else:
        same_codes_unique_mdd_removed.append(i)

In [20]:
not_in_dataset_unique_mdd_removed = []
for i in not_in_dataset_unique:
    if np.array(data[data.code == i].FG).shape[0] > 1:
        fg_i = np.array(data[data.code == i].FG)
        bpe_i = np.array(data[data.code == i].BPE)
        if np.all(fg_i==fg_i[0]) and np.all(bpe_i==bpe_i[0]):
            not_in_dataset_unique_mdd_removed.append(i)
    else:
        not_in_dataset_unique_mdd_removed.append(i)
        

In [21]:
all_ok_data = same_codes_unique_mdd_removed + not_in_dataset_unique_mdd_removed

In [22]:
len(all_ok_data)

1146

# Part 1


In [23]:
mpath = 'H:/Breast MRI/01/sps'
folder_names_part1 = []
folder_names_part1 = get_folder_names(mpath, folder_names_part1)

In [24]:
for name in folder_names_part1:
    if name not in all_ok_data:
        print(name)

In [25]:
no_data_cases = ['9222704', '9999465', '8881092', '11258955', '11843612']

In [26]:
len(all_ok_data)

1146

In [27]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [28]:
len(all_ok_data)

1142

In [29]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/01/sps'

In [30]:
patients_ids_1 = [8860408, 8861603, 8861971, 8866708, 8889836, 8915593, 8915632, 8926218, 8934757, 8935156, 8939476, 8977117,
                  9008081, 9017845, 9108321, 9162757, 9170060, 9173422, 9174037, 9182263, 9221819, 9239385, 9246035, 9354593,
                  9362401, 9363679, 9410575, 9459081, 9459302, 9459663, 9471658, 9471667, 9471756, 9471997, 9474758, 9484852,
                  9484938, 9486031, 9486352, 9492065, 9512135, 9535197, 9535278, 9535434, 9559948, 9571865, 9577964, 9578077,
                  9638156, 9638749, 9646672, 9647040, 9649691, 9658124, 9661700, 9670311, 9670327, 9673120, 9673179, 9673778,
                  9674663, 9680775, 9680867, 9700791, 9701051, 9718836, 9726839, 9727663, 9739752, 9757971, 9773985, 9852342,
                  9900364, 9906687, 9906752, 9910509, 9918129, 9918268, 9918324, 9920466, 9924862, 9926215, 9929361, 9929533,
                  9930243, 9953872, 9953955, 9954110, 9966610, 9970531, 9971081, 9975377, 9975397, 9985427, 9991646, 9994490,
                  9997801, 9997958, 9998057, 10006533, 10007023, 10007991, 10010860, 10018165, 10030277, 10032605, 10032779,
                  10045260, 11201493, 11201533, 11205832, 11208303, 11211696, 11218656, 11224788, 11231020, 11237684, 11239822,
                  11244816, 11247503, 11254075, 11256378, 11264398, 11272908, 11273475, 11286927, 11290544, 11299936, 11301332,
                  11306141, 11331505, 11333648, 11333741, 11338126, 11338192, 11342837, 11344516, 11348359, 11348891, 11351690,
                  11352729, 11355812, 11356851, 11358818, 11366628, 11366836, 11367575, 11369417, 11369686, 11371493, 11372703,
                  11375497, 11377087, 11379700]

In [31]:
fgt_1 = [(300, 451), (373, 458), (558, 685), (396, 481), (536, 621), (379, 550), (434, 517), (335, 478), (582, 661), (587, 662),
         (303, 454), (348, 523), (533, 618), (365, 440), (514, 599), (335, 510), (405, 548), (589, 674), (505, 590), (348, 499),
         (521, 606), (387, 542), (407, 492), (554, 639), (365, 450), (365, 450), (432, 513), (490, 567), (371, 442), (464, 549),
         (365, 524), (365, 528), (422, 573), (338, 485), (591, 676), (493, 668), (393, 560), (432, 587), (595, 758), (325, 448),
         (365, 528), (365, 438), (302, 367), (365, 430), (365, 438), (501, 586), (569, 654), (529, 614), (277, 356), (277, 352),
         (281, 366), (249, 314), (490, 575), (420, 505), (277, 352), (536, 679), (277, 352), (421, 506), (474, 559), (439, 524), 
         (441, 526), (471, 560), (315, 406), (449, 536), (421, 510), (466, 551), (332, 499), (269, 354), (483, 518), (479, 650), 
         (461, 546), (435, 520), (552, 735), (431, 518), (443, 530), (423, 586), (402, 545), (485, 628), (380, 523), (430, 515), 
         (496, 671), (597, 800), (385, 464), (451, 540), (299, 384), (313, 404), (315, 402), (317, 404), (435, 520), (448, 639), 
         (612, 771), (439, 526), (517, 602), (500, 585), (405, 490), (555, 640), (426, 513), (297, 382), (277, 350), (506, 649), 
         (464, 549), (454, 539), (455, 598), (471, 556), (435, 598), (407, 554), (426, 577), (299, 384), (467, 662), (421, 612), 
         (376, 551), (258, 343), (275, 450), (429, 620), (337, 512), (500, 675), (350, 525), (396, 575), (406, 581), (405, 490), 
         (380, 555), (413, 604), (393, 568), (417, 592), (383, 558), (423, 508), (404, 579), (417, 640), (461, 604), (402, 577), 
         (286, 461), (359, 534), (587, 762), (399, 574), (385, 560), (420, 607), (370, 545), (417, 502), (405, 490), (395, 570), 
         (552, 727), (363, 538), (567, 718), (369, 544), (407, 582), (415, 602), (393, 568), (426, 601), (385, 560), (389, 564), 
         (344, 519), (355, 530), (428, 619), (561, 756)]

In [32]:
bpe_1 = [(1060, 1211), (920, 1005), (2589, 2716), (914, 999), (1162, 1247), (1615, 1786), (1042, 1125), (1235, 1378), (1054, 1133),
         (1035, 1110), (1243, 1394), (1408, 1583), (1071, 1156), (813, 888), (1022, 1107), (1395, 1570), (1349, 1492), (1157, 1242),
         (1061, 1146), (1288, 1439), (1047, 1132), (1323, 1478), (925, 1010), (1080, 1165), (886, 971), (915, 1000), (928, 1009),
         (978, 1055), (820, 891), (980, 1065), (1233, 1392), (1253, 1416), (1252, 1403), (1146, 1293), (1141, 1226), (1441, 1616),
         (1301, 1468), (1280, 1435), (1699, 1862), (1225, 1348), (1253, 1416), (803, 876), (700, 765), (763, 828), (803, 876),
         (1007, 1092), (1107, 1192), (1063, 1148), (769, 848), (749, 824), (803, 888), (671, 736), (1032, 1117), (954, 1039),
         (749, 824), (2267, 2410), (749, 824), (959, 1044), (1012, 1097), (977, 1062), (979, 1064), (1025, 1114), (889, 980), 
         (1002, 1089), (978, 1067), (1015, 1100), (1238, 1405), (783, 868), (1203, 1288), (2382, 2553), (1126, 1211), (969, 1054), 
         (1872, 2055), (984, 1071), (990, 1077), (1423, 1586), (1278, 1421), (1373, 1516), (1256, 1399), (1228, 1313), (1768, 1943), 
         (2001, 2204), (869, 948), (1011, 1100), (849, 934), (880, 971), (868, 955), (871, 958), (957, 1042), (1816, 2007), 
         (2760, 2919), (971, 1058), (1150, 1235), (1155, 1240), (927, 1012), (1227, 1312), (1065, 1152), (834, 919), (754, 827), 
         (1226, 1369), (1161, 1246), (1107, 1192), (1487, 1630), (1108, 1193), (1411, 1574), (1303, 1450), (1342, 1493), (845, 930), 
         (1859, 2054), (1813, 2004), (1648, 1823), (792, 877), (1511, 1686), (1833, 2024), (1579, 1754), (1736, 1911), (1610, 1785), 
         (1716, 1895), (1646, 1821), (936, 1021), (1616, 1791), (1793, 1984), (1635, 1810), (1725, 1900), (1643, 1818), (957, 1042), 
         (1820, 1995), (2723, 2946), (1337, 1480), (1746, 1921), (1522, 1697), (1667, 1842), (1859, 2034), (1671, 1846), (1657, 1832), 
         (1788, 1975), (1610, 1785), (948, 1033), (939, 1024), (1631, 1806), (1788, 1963), (1683, 1858), (1327, 1478), (1605, 1780), 
         (1651, 1826), (1731, 1918), (1635, 1810), (1914, 2089), (1657, 1832), (1673, 1848), (1580, 1755), (1651, 1826), (1820, 2011),
         (1977, 2172)]

In [33]:
# create_pid_folder(d_directory, patients_ids_1)

In [34]:
# automated_copy(o_directory, d_directory, patients_ids_1, fgt_1, bpe_1)

# Part 2


In [35]:
mpath = 'H:/Breast MRI/02/sps'
folder_names_part2 = []
folder_names_part2 = get_folder_names(mpath, folder_names_part2)

In [36]:
for name in folder_names_part2:
    if name not in all_ok_data:
        print(name)

In [37]:
no_data_cases = ['1191379', '8368349', '8832712', '8145891', '8302127', '8305735', '8414121', '8675439', '8675610', '8682515']

In [38]:
len(all_ok_data)

1142

In [39]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [40]:
len(all_ok_data)

1135

In [41]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [42]:
patients_ids_2 = []

In [43]:
fgt_2 = []

In [44]:
bpe_2 = []

In [45]:
# create_pid_folder(d_directory, patients_ids_2)

In [46]:
# automated_copy(o_directory, d_directory, patients_ids_2, fgt_2, bpe_2)

# Part 3


In [47]:
mpath = 'H:/Breast MRI/03/breast new'
folder_names_part3 = []
folder_names_part3 = get_folder_names(mpath, folder_names_part3)

In [48]:
for name in folder_names_part3:
    if name not in all_ok_data:
        print(name)

In [49]:
no_data_cases = []

In [50]:
len(all_ok_data)

1135

In [51]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [52]:
len(all_ok_data)

1135

In [53]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [54]:
patients_ids_3 = []

In [55]:
fgt_3 = []

In [56]:
bpe_3 = []

In [57]:
# create_pid_folder(d_directory, patients_ids_3)

In [58]:
# automated_copy(o_directory, d_directory, patients_ids_3, fgt_3, bpe_3)

# Part 4


In [59]:
mpath = 'H:/Breast MRI/03/breast new 2'
folder_names_part4 = []
folder_names_part4 = get_folder_names(mpath, folder_names_part4)

In [60]:
folder_list = []
for name in folder_names_part4:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [61]:
len(folder_list)

0

In [62]:
# source_directory = mpath
# destination_directory = "H:/Breast MRI/03/NotOkData/breast new 2"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [63]:
no_data_cases = ['10942269']

In [64]:
len(all_ok_data)

1135

In [65]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [66]:
len(all_ok_data)

1134

In [67]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [68]:
patients_ids_4 = []

In [69]:
fgt_4 = []

In [70]:
bpe_4 = []

In [71]:
# create_pid_folder(d_directory, patients_ids_4)

In [72]:
# automated_copy(o_directory, d_directory, patients_ids_4, fgt_4, bpe_4)

# Part 5


In [73]:
mpath = 'H:/Breast MRI/03/nac'
folder_names_part5 = []
folder_names_part5 = get_folder_names(mpath, folder_names_part5)

In [74]:
folder_list = []
for name in folder_names_part5:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [75]:
len(folder_list)

0

In [76]:
# source_directory = mpath
# destination_directory = "H:/Breast MRI/03/NotOkData/nac"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [77]:
no_data_cases = []

In [78]:
len(all_ok_data)

1134

In [79]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [80]:
len(all_ok_data)

1134

In [81]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [1]:
patients_ids_5 = [10500639, 10506817, 10507162, 10509355, 10520523, 10532430, 10540595, 10553401, 10554186, 10561171, 10581017, 10591458, 
                10632327, 10716690, 10718503, 10738155, 10743894, 10756274, 10773065, 10791568, 10795621, 10812027, 10879476, 10905568, 
                10916400, 10937977, 10959273, 10983337, 11032357, 11034705, 11075626, 11076778, 11086355, 11180494, 11183489, 11216685,
                11275661, 11346010, 11627699]

In [83]:
fgt_5 = [(941,1132), (982,1173), (1287,1478), (1017,1208), (1190,1389), (1033,1224),
 (991,1182), (1052,1243), (1160,1351), (978,1169),(879,1038), (1027,1218), (1224,1415), (974,1153), (1694,1885),
 (1134,1325), (1155,1346), (1174,1365), (1174,1365), (1250,1441), (405,490), (1181,1372), (1038,1229), (1007,1194),
 (631,806), (963,1154), (536,679), (1141,1332), (2281,2484), (1865,2068), (867,1006), (1204,1395), (1246,1437),
 (1177,1380), (998,1189), (916,1107), (1397,1576), (896,1087)
 ]

In [84]:
bpe_5 = [(2285,2479), (2416,2607), (3199,3390), (2451,2642), (3269,3468), (2577,2768), (2941,3132), (2558,2749), (2688,2879), (2928,3119), (2687,2846), 
(2571,2762), (2776,2967), (2430,2609), (3250,3441), (2688,2879),(2567,2758), (2718,2909), (2729,2920), (2824,3015), (957,1042), (2785,2976), (2582,2773),
(2523,2710), (1915,2090), (2507,2698), (2116,2259), (3200,3391), (4025,4228), (3648,3851), (2033,2172), (2768,2959), (2796,2987), (2810,3013), (2542,2733),
(2881,3092), (2460,2651), (2849,3028), (2440,2631)
]

In [85]:
# create_pid_folder(d_directory, patients_ids_5)

In [86]:
# automated_copy(o_directory, d_directory, patients_ids_5, fgt_5, bpe_5)

# Part 6


In [87]:
mpath = 'H:/Breast MRI/03/sps'
folder_names_part6 = []
folder_names_part6 = get_folder_names(mpath, folder_names_part6)

In [88]:
folder_list = []
for name in folder_names_part6:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [89]:
len(folder_list)

0

In [90]:
# source_directory = mpath
# destination_directory = "H:/Breast MRI/03/NotOkData/sps"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [91]:
no_data_cases = ['8390792', '8564813', '8613792', '8756913']

In [92]:
len(all_ok_data)

1134

In [93]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [94]:
len(all_ok_data)

1130

In [95]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [96]:
patients_ids_6 = [8308544, 8320097, 8644677, 8654723, 8668965, 8693340, 8845788, 8846009, 8943630, 8980696, 8987973, 8994596, 8995280, 9002280, 9010579, 9010854, 9011738, 9012992, 9013052, 9017038, 9021496, 9024061, 9024191, 9024288, 9024861, 9064702,
9066182, 9072270, 9075718, 9075921, 9082292, 9099448, 9099484, 9099535, 9102847, 9103559, 9103635, 9124636, 9132962, 9135746, 9137960, 9178082, 9186604, 9187014, 9211330, 9221344, 9246145, 9250562,
9250859, 9251259, 9257467, 9260607, 9278242, 9278242, 9278286, 9278507, 9280993, 9282859, 9297184, 9300041, 9303450, 9303958, 9306355, 9314842, 9314854, 9329696, 9329801, 9338697, 9338721, 9350416, 9354498, 9374330, 9374338, 9374628,
9376635, 9378242, 9380740, 9385156, 9385880, 9397693, 9397802, 9398002, 9400054, 9401113, 9403123, 9407013, 9408592, 9409404, 9433277, 9433866, 9434346, 9446950, 9447103, 9449885, 9452045, 9456472, 9456991, 9555999, 9571196, 9590052,
9613321, 9613957, 9634850, 9657991, 9670407, 9679589, 9683733, 9692694, 9692981, 9730700, 9749506
]

In [97]:
fgt_6 = [(364,515), (658,833), (356,413), (321,472), (536,679), (536,623), (348,523), (345,512), (356,531), (518,615), (348,523), (387,472), (396,481), (338,501), (605,690), (622,707), (340,423), (431,516), (365,450), (387,472), (269,354), (365,450), (509,594), (365,450), (505,590), (454,625),
(335,502), (348,523), (549,634), (525,610), (545,688), (509,594), (378,463), (482,653), (625,710), (521,606), (580,665), (529,614), (320,471), (430,561), (344,479), (427,512), (365,442), (378,453), (350,517), (509,594), (421,506), (394,479),
(397,482), (403,488), (661,746), (409,494), (332,475), (397,540), (371,522), (411,562), (568,653), (335,498), (421,506), (393,536), (368,511), (809,894), (393,478), (467,556), (551,694), (365,430), (439,524), (450,535), (365,450), (451,536), (537,622), (427,512), (365,450), (330,481),
(549,634), (529,614), (375,550), (365,430), (387,474), (399,542), (413,572), (391,460), (541,626), (597,682), (521,606), (373,458), (365,432), (401,486), (517,602), (521,606), (633,718), (377,528), (348,503), (554,639), (574,659), (417,502), (411,496), (509,594), (489,574), (396,481), 
(403,472), (275,350), (301,394), (405,470), (277,352), (273,348), (415,504), (459,544), (505,590), (519,604), (279,344)
]

In [98]:
bpe_6 = [ (1124,1275), (1606,1781), (782,839), (1081,1232), (2092,2235), (1056,1143), (1408,1583), (1245,1412), (1416,1591), (1206,1303), (1408,1583), (885,970), (894,979), (1338,1501), (1103,1188), (1124,1209), (848,931), (1317,1402), (1262,1347), (971,1056), (699,784), (1247,1332), (1458,1543), (1126,1211), (1015,1100), (1554,1725),
(1523,1690), (1408,1583), (1150,1235), (1081,1166), (1601,1744), (1151,1236), (876,961), (1410,1581), (1205,1290), (1083,1168), (1198,1283), (1105,1190), (1236,1387), (1402,1533), (1340,1475), (1102,1187), (823,900), (826,901), (1370,1537), (1023,1108), (939,1024), (1123,1208),
(1202,1287), (1244,1239), (1211,1296), (951,1036), (1208,1351), (1273,1416), (1287,1438), (1375,1526), (1106,1191), (1335,1498), (1101,1186), (1269,1412), (1244,1387), (1515,1600), (904,989), (1228,1317), (1427,1570), (763,828), (981,1066), (948,1033), (1101,1186), (1209,1294), (1079,1164), (925,1010), (863,948), (1270,1421),
(1111,1196), (1063,1148), (1435,1610), (1089,1154), (908,995), (1275,1418), (1369,1528), (809,878), (1087,1172), (1135,1220), (1047,1132), (807,992), (1126,1193), (914,999), (1039,1124), (1047,1132), (1179,1264), (1293,1444), (1308,1463), (1080,1165), (1120,1205), (943,1028), (933,1018), (1242,1327), (1256,1341), (937,1022),
(845,914), (747,822), (863,956), (842,907), (749,824), (745,820), (969,1058), (1136,1221), (1170,1255), (1073,1158), (725,796)
]

In [99]:
# create_pid_folder(d_directory, patients_ids_1)

In [100]:
# automated_copy(o_directory, d_directory, patients_ids_6, fgt_6, bpe_6)

# Part 7


In [101]:
mpath = "E:/Breast Data/Breast"
folder_names_part7 = []
folder_names_part7 = get_folder_names(mpath, folder_names_part7)

In [102]:
folder_list = []
for name in folder_names_part7:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [103]:
len(folder_list)

0

In [104]:
# source_directory = mpath
# destination_directory = "E:/Breast Data/NotOkData/Breast"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [105]:
no_data_cases = ['11821274']

In [106]:
len(all_ok_data)

1130

In [107]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [108]:
len(all_ok_data)

1129

In [109]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [110]:
patients_ids_7 = []

In [111]:
fgt_7 = []

In [112]:
bpe_7 = []

In [113]:
# create_pid_folder(d_directory, patients_ids_7)

In [114]:
# automated_copy(o_directory, d_directory, patients_ids_7, fgt_7, bpe_7)

# Part 8


In [115]:
mpath = "E:/Breast Data/breast newNAC"
folder_names_part8 = []
folder_names_part8 = get_folder_names(mpath, folder_names_part8)

In [116]:
folder_list = []
for name in folder_names_part8:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [117]:
len(folder_list)

0

In [118]:
# source_directory = mpath
# destination_directory = "E:/Breast Data/NotOkData/breast newNAC"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [119]:
no_data_cases = ['10905568', '10937977']

In [120]:
len(all_ok_data)

1129

In [121]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [122]:
len(all_ok_data)

1127

In [123]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [124]:
patients_ids_8 = []

In [125]:
fgt_8 = []

In [126]:
bpe_8 = []

In [127]:
# create_pid_folder(d_directory, patients_ids_1)

In [128]:
# automated_copy(o_directory, d_directory, patients_ids_1, fgt_1, bpe_1)

# Part 9


In [129]:
mpath = "G:/Breast Data/nac"
folder_names_part9 = []
folder_names_part9 = get_folder_names(mpath, folder_names_part9)

In [130]:
folder_list = []
for name in folder_names_part9:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [131]:
len(folder_list)

0

In [132]:
# source_directory = mpath
# destination_directory = "G:/Breast Data/NotOkData/nac"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [133]:
no_data_cases = []

In [134]:
len(all_ok_data)

1127

In [135]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [136]:
len(all_ok_data)

1127

In [137]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [138]:
patients_ids_9 = []

In [139]:
fgt_9 = []

In [140]:
bpe_9 = []

In [141]:
# create_pid_folder(d_directory, patients_ids_9)

In [142]:
# automated_copy(o_directory, d_directory, patients_ids_9, fgt_9, bpe_9)

# Part 10


In [156]:
mpath = "G:/Breast Data/sps"
folder_names_part10 = []
folder_names_part10 = get_folder_names(mpath, folder_names_part10)

In [157]:
folder_list = []
for name in folder_names_part10:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [158]:
len(folder_list)

0

In [159]:
# source_directory = mpath
# destination_directory = "G:/Breast Data/NotOkData/sps"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [160]:
no_data_cases = ['8867592']

In [161]:
len(all_ok_data)

1127

In [162]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [163]:
len(all_ok_data)

1126

In [137]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [138]:
patients_ids_10 = []

In [139]:
fgt_10 = []

In [140]:
bpe_10 = []

In [141]:
# create_pid_folder(d_directory, patients_ids_10)

In [142]:
# automated_copy(o_directory, d_directory, patients_ids_10, fgt_10, bpe_10)